# <ins> Results Mapping Notebook <ins/>
> By: Matteo Cangiano, Q Leedham, and Daniel Sheehan

In [2]:
import pandas as pd
import geopandas

In [124]:
df = pd.read_csv('../data/predicted-favelas.csv')  # read in predictions
df['cd_geocodi'] = df['cd_geocodi'].astype(str) 

In [125]:
wd = '/Users/danielmsheehan/Dropbox/General_Assembly/Projects/project_5/data'  # working file directory, see Input Datasets Geoprocessing.ipynb

geo_df = geopandas.read_file(f'{wd}/maps/sp_census_tracts_wgs_84.shp')
geo_df.columns = [c.lower() for c in geo_df.columns]
geo_df = geo_df[['cd_geocodi', 'geometry']]
geo_df = geo_df.merge(df, on='cd_geocodi', how='left')

true_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 1) ]
false_neg = geo_df[(geo_df['favela_present'] == 0) & (geo_df['predicted_favela_present'] == 1) ]
false_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 0) ]

true_pos_geojson = true_pos.to_crs(epsg='4326').to_json()  # create geojson for maps
false_neg_geojson = false_neg.to_crs(epsg='4326').to_json()
false_pos_geojson = false_pos.to_crs(epsg='4326').to_json()

true_pos.to_file(f'{wd}/maps/favela_preds_true_pos.shp')  # create shapefiles
false_neg.to_file(f'{wd}/maps/favela_preds_false_neg.shp')
false_pos.to_file(f'{wd}/maps/favela_preds_false_pos.shp')

In [126]:
real_favela = geopandas.read_file(f'{wd}/maps/sp_favelas_wgs_84.shp')  # get favelas
real_favela_geojson = real_favela.to_crs(epsg='4326').to_json()

# Predictions Map

In [130]:
import folium

m = folium.Map(location=[-23.55, -46.533333], tiles="cartodbpositron", zoom_start=12)

def style_true_pos_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#66ff99'
    }

def style_false_neg_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#cc0099'
    }

def style_false_pos_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 0,
        'fillColor': '#ffcc00'
    }

def style_real_favela_geojson(feature):
    return {
        #'fillOpacity': 0.9,
        'weight': 4,
        'fillColor': '#66ff99',
        'color': '#001a09',
    }


folium.GeoJson(
    real_favela_geojson, 
    name='Real Favelas', 
    style_function=style_real_favela_geojson,
).add_to(m)


folium.GeoJson(
    true_pos_geojson, 
    name='True Positives', 
    style_function=style_true_pos_geojson,
).add_to(m)

folium.GeoJson(
    false_neg_geojson, 
    name='False Negatives',
    style_function=style_false_neg_geojson,
).add_to(m)

folium.GeoJson(
    false_pos_geojson, 
    name='False Positives',
    style_function=style_false_pos_geojson,
).add_to(m)



folium.LayerControl().add_to(m)

m.save('html/index.html')

![](https://raw.git.generalassemb.ly/delta/delta/master/geoprocessing/images/final_predictions_map.png?token=AABhyVeWXSK5OknmwYuJg0btIRko_N_gks5eWXC9wA%3D%3D)
_Output image embedded in case viewing on GitHub_

Or visit [https://predictfavelas.github.io/](https://predictfavelas.github.io/)

# Data Map

In [132]:
census_tracts = geopandas.read_file(f'{wd}/maps/sp_census_tracts_select_example_wgs_84.shp')
census_tracts_geojson = census_tracts.to_crs(epsg='4326').to_json()

favelas = geopandas.read_file(f'{wd}/maps/sp_favelas_wgs_84.shp')
favelas_geojson = favelas.to_crs(epsg='4326').to_json()


m = folium.Map(location=[-23.65, -46.59033333], tiles="cartodbpositron", zoom_start=12)


def style_census_tracts_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 4,
        'fillColor': '#3385ff',
        'color': '#002966',
    }

def style_favelas_geojson(feature):
    return {
        'fillOpacity': 0.8,
        'weight': 4,
        'fillColor': '#66ff99',
        'color': '#001a09',
    }

folium.GeoJson(
    census_tracts_geojson, 
    name='Census Tracts Example', 
    style_function=style_census_tracts_geojson,
).add_to(m)


folium.GeoJson(
    favelas_geojson, 
    name='Real Favelas', 
    style_function=style_favelas_geojson,
).add_to(m)

folium.LayerControl().add_to(m)

m.save('html/data.html')

![](https://raw.git.generalassemb.ly/delta/delta/master/geoprocessing/images/example_data_map.png?token=AABhyVDwSlkbvlB2QDD5BTp36o4mf5hrks5eWXDZwA%3D%3D)
_Output image embedded in case viewing on GitHub_

Or visit [https://predictfavelas.github.io/data](https://predictfavelas.github.io/data)

# Map for Kumar's Predictions

predicted-favelas-kumar.csv

In [1]:
df = pd.read_csv('../data/predicted-favelas-kumar.csv')  # read in predictions
df['cd_geocodi'] = df['cd_geocodi'].astype(str) 

NameError: name 'pd' is not defined

In [3]:
wd = '/Users/danielmsheehan/Dropbox/General_Assembly/Projects/project_5/data'  # working file directory, see Input Datasets Geoprocessing.ipynb

geo_df = geopandas.read_file(f'{wd}/maps/sp_census_tracts_wgs_84.shp')
geo_df.columns = [c.lower() for c in geo_df.columns]
geo_df = geo_df[['cd_geocodi', 'geometry']]
geo_df = geo_df.merge(df, on='cd_geocodi', how='left')

true_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 1) ]
false_neg = geo_df[(geo_df['favela_present'] == 0) & (geo_df['predicted_favela_present'] == 1) ]
false_pos = geo_df[(geo_df['favela_present'] == 1) & (geo_df['predicted_favela_present'] == 0) ]

true_pos_geojson = true_pos.to_crs(epsg='4326').to_json()  # create geojson for maps
false_neg_geojson = false_neg.to_crs(epsg='4326').to_json()
false_pos_geojson = false_pos.to_crs(epsg='4326').to_json()

true_pos.to_file(f'{wd}/maps/favela_preds_true_pos.shp')  # create shapefiles
false_neg.to_file(f'{wd}/maps/favela_preds_false_neg.shp')
false_pos.to_file(f'{wd}/maps/favela_preds_false_pos.shp')

NameError: name 'df' is not defined

In [ ]:
real_favela = geopandas.read_file(f'{wd}/maps/sp_favelas_wgs_84.shp')  # get favelas
real_favela_geojson = real_favela.to_crs(epsg='4326').to_json()